<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo 5 Exercício I** | Tratamento de Dados
**Exercícios**<br> 
[Luiz Aires](https://github.com/luizaires)

---

# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

---

In [1]:
#Parte 1-Carregando base de dados SINASC-RO
import pandas as pd
import requests

sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


---

In [3]:
# Parte 2-número de valores NaN por variável
sinasc.isna().sum().sort_values(ascending = False)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DTCADASTRO        0
NUMEROLOTE        0
VERSAOSIST        0
ORIGEM            0
Length: 69, dtype: int64

---

In [4]:
# Parte 3-Atribuindo um novo Data Frame apenas com as varáveis de interesse
sinasc = sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                          'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [5]:
#Número de missings para cada variável no Data Frame de interesse
sinasc.isna().sum().sort_values(ascending = False)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

---

In [6]:
# Parte 4-Removendo as linhas que não possuem dados para APGAR5
sinasc = sinasc[~sinasc['APGAR5'].isna()]
#(nº de linhas, nº de colunas)
print(sinasc.shape)
#Nº de missings para cada variável
sinasc.isna().sum().sort_values(ascending = False) 

(26925, 9)


QTDFILVIVO    1566
GESTACAO      1216
ESTCIVMAE      315
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
APGAR5           0
dtype: int64

---

In [7]:
# Parte 5-Tratando dados ausentes nas colunas 'ESTCIVMAE'
#Substituindo missings por 9
sinasc['ESTCIVMAE'].fillna(9, inplace = True)
#Verificando o número de NaN, após uso do método fillna
sinasc['ESTCIVMAE'].isna().sum()

0

---

In [8]:
# Parte 6-Substituindo valores ausentes para a variável 'QTDFILVIVO' por zero
sinasc['QTDFILVIVO'].fillna(0, inplace = True)
sinasc['QTDFILVIVO'].isna().sum()

0

---

In [9]:
# Parte 7-Tratando dados ausentes para as variáveis:
#Escolaridade da mãe
sinasc['ESCMAE'].fillna(9, inplace = True)
#Semanas de gestação
sinasc['GESTACAO'].fillna(9, inplace = True)
#Como as varáveis representam categorias,optou-se por substituir Nan por 9 (Não preenchido).

In [10]:
#Tipo de gravidez
#Optou-se por substituir os valores ausentes pelo valor mais frequente
print(sinasc['GRAVIDEZ'].value_counts()/26925)
sinasc['GRAVIDEZ'].fillna('Única', inplace = True)

Única              0.978867
Dupla              0.018124
Tríplice e mais    0.000186
Name: GRAVIDEZ, dtype: float64


---

In [11]:
# Parte 8-Categorização da variável 'APGAR5'
sinasc.loc[(sinasc['APGAR5'] >= 8) & (sinasc['APGAR5'] <= 10), 'class_APGAR5'] = 'Normal'
sinasc.loc[(sinasc['APGAR5'] >= 6) & (sinasc['APGAR5'] <= 7), 'class_APGAR5'] = 'Asfixia Leve'
sinasc.loc[(sinasc['APGAR5'] >= 4) & (sinasc['APGAR5'] <= 5), 'class_APGAR5'] = 'Asfixia Moderada'
sinasc.loc[(sinasc['APGAR5'] >= 0) & (sinasc['APGAR5'] <= 3), 'class_APGAR5'] = 'Asfixia Severa'
sinasc['class_APGAR5'].unique()

array(['Normal', 'Asfixia Leve', 'Asfixia Severa', 'Asfixia Moderada'],
      dtype=object)

---

In [12]:
# Parte 9-Renomeando os valores da coluna 'class_APGAR5' para que fiquem em snake case
sinasc['class_APGAR5'] = sinasc['class_APGAR5'].str.replace(' ', '_').str.lower()
sinasc['class_APGAR5'].unique()

array(['normal', 'asfixia_leve', 'asfixia_severa', 'asfixia_moderada'],
      dtype=object)

---